In [1]:
from openai import OpenAI

openai_client = OpenAI()

In [5]:
import json

from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import AppendableIndex


reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

In [6]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the documentation database for relevant results based on a query string.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to look up in the index"
            }
        },
        "required": [
            "query"
        ]
    }
}

In [65]:
def add_entry(filename, title, description, content):
    entry = {
        'start': 0,
        'content': content,
        'title': title,
        'description': description,
        'filename': filename,
    }
    index.append(entry)
    return "OK"

add_entry_tool = {
    "type": "function",
    "name": "add_entry",
    "description": "Add a new documentation entry to the index.",
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "The source filename associated with the entry"
            },
            "title": {
                "type": "string",
                "description": "The title of the documentation entry"
            },
            "description": {
                "type": "string",
                "description": "A short description summarizing the entry"
            },
            "content": {
                "type": "string",
                "description": "The full content of the documentation entry"
            }
        },
        "required": [
            "filename",
            "title",
            "description",
            "content"
        ]
    }
}


In [66]:
def make_call(tool_call):
    arguments = json.loads(tool_call.arguments)
    name = tool_call.name

    if name == 'search':
        result = search(**arguments)
    elif name == 'add_entry':
        result = add_entry(**arguments)
    else: 
        result = 'not found tool "{name}"'
    
    return {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": json.dumps(result),
    }

In [59]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

Make 3 iterations:

1) in the first iteration, perform one search
2) in the second interation, analyze the results from the previous search
   and perform 2 more searches
3) synthesise the results into the output

IMPORTANT: at each step, give an explanation of why you want to perform 
search for this particular search query. It should be 2-3 sentences explaining
the logic of your decision.

Use only facts from the knowledge base when answering.
If you cannot find the answer, inform the user.

Our knowledge base is entirely about Evidently, so you don't need to 
include the word 'evidently' in search results
"""

In [60]:
question = "How do I create a dahsbord in Evidently?"

In [74]:
tools = [search_tool, add_entry_tool]

message_history = [
    {"role": "system", "content": instructions},
]

iteration_number = 1

# Q&A loop
while True:
    user_prompt = input('You:')
    if user_prompt.lower().strip() == 'stop':
        break

    message_history.append({"role": "user", "content": user_prompt})

    # tool-call loop
    while True:
        response = openai_client.responses.create(
            model='gpt-4o-mini',
            input=message_history,
            tools=tools,
        )
    
        print(f'iteraration number {iteration_number}...') 
        message_history.extend(response.output)
    
        has_function_calls = False
    
        for message in response.output:
            if message.type == 'function_call':
                print(f'executing {message.name}({message.arguments})...')
                tool_call_output = make_call(message)
                message_history.append(tool_call_output)
                has_function_calls = True
    
            if message.type == 'message':
                text = message.content[0].text
                print('ASSISTANT:', text)
    
        iteration_number = iteration_number + 1
        print()
        
        if not has_function_calls:
            break

You: dahsbord design


iteraration number 1...
ASSISTANT: To start, I'll search for "dashboard design" to find relevant documentation about creating and designing dashboards. Understanding the principles and best practices involved in dashboard design is crucial for delivering meaningful data visualizations and user experiences. 

Let's see what I can find.
executing search({"query":"dashboard design"})...

iteraration number 2...
ASSISTANT: The first search results provided information about adding panels to a dashboard, including necessary commands and examples. This gives a foundational understanding of how to structure and customize a dashboard using various panel types like text, counters, and charts.

Now, I'll refine my search to dig deeper into specific aspects of dashboard design. I will search for "dashboard customization options" to explore how to enhance the visual appeal and functional aspects of dashboards, ensuring they meet user needs effectively. Additionally, I will look for "best practices

You: stop


In [78]:
class Agent:

    def __init__(self, llm_client, model, instructions, tools):
        self.llm_client = llm_client
        self.model = model
        self.instructions = instructions
        self.tools = tools

    def make_call(self, tool_call):
        arguments = json.loads(tool_call.arguments)
        name = tool_call.name
    
        if name == 'search':
            result = search(**arguments)
        elif name == 'add_entry':
            result = add_entry(**arguments)
        else:
            result = 'not found tool "{name}"'
        
        return {
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": json.dumps(result),
        }   

    def loop(self, user_prompt, message_history=None):
        if not message_history:
            message_history = [
                {"role": "system", "content": self.instructions},
            ]
            
        message_history.append({"role": "user", "content": user_prompt})

        iteration_number = 0
    
        while True:
            response = self.llm_client.responses.create(
                model=self.model,
                input=message_history,
                tools=self.tools,
            )
        
            print(f'iteraration number {iteration_number}...') 
            message_history.extend(response.output)
        
            has_function_calls = False
        
            for message in response.output:
                if message.type == 'function_call':
                    print(f'executing {message.name}({message.arguments})...')
                    tool_call_output = self.make_call(message)
                    message_history.append(tool_call_output)
                    has_function_calls = True
        
                if message.type == 'message':
                    text = message.content[0].text
                    print('ASSISTANT:', text)
        
            iteration_number = iteration_number + 1
            print()
            
            if not has_function_calls:
                break

        return message_history        

    def qna(self):
        message_history = [
            {"role": "system", "content": instructions},
        ]
        
        iteration_number = 1

        # Q&A loop
        while True:
            user_prompt = input('You:')
            if user_prompt.lower().strip() == 'stop':
                break
            
            message_history = self.loop(user_prompt, message_history)

In [79]:
agent = Agent(
    llm_client=OpenAI(),
    model='gpt-4o-mini',
    instructions=instructions,
    tools=tools
)

In [80]:
messages = agent.loop('evidently adshboards') 

iteraration number 0...
ASSISTANT: I will first perform a search for "dashboards" to gather information on how this feature operates and any relevant details surrounding its usage within the system. This will help me understand the foundational aspects of dashboards and their functionalities.

Let's proceed with the search.
executing search({"query":"dashboards"})...

iteraration number 1...
ASSISTANT: From the initial search results, it appears that dashboards serve as visual tools for tracking evaluation results, featuring panels that can be customized to display various metrics over time. These dashboards can be created both through a user interface and a Python API. There are multiple options available for organizing panels, such as tabs and types of visualizations (e.g., counters, pie charts, line plots).

To gain a more in-depth understanding of how to customize these dashboards and the types of panels available, I will conduct two additional searches: one focused on "customizing

In [81]:
agent.qna()

You: adshboad evidently


iteraration number 0...
ASSISTANT: To better understand the concept of "dashboard," I'll perform a search specifically for "dashboard." This will help me find documentation related to the features, functionalities, and related components of dashboards within the context of the platform.

Let's start with this search.
executing search({"query":"dashboard"})...

iteraration number 1...
ASSISTANT: The initial search on "dashboard" yielded content primarily about how to manage and add panels to a dashboard via the API. The results provide detailed instructions on adding tabs and panels, including various types of panels like text panels, counters, and charts. This gives a foundational understanding of how dashboards are structured and how users can customize them.

To deepen my understanding, I want to perform two more searches: one on "dashboard customization" to explore additional options for personalizing dashboards, and another on "dashboard metrics" to understand how to incorporate va

You: add this to the database


iteraration number 0...
executing add_entry({"filename":"docs/platform/dashboard_overview.mdx","title":"Dashboard Overview","description":"Comprehensive overview of dashboard features, customization options, and panel management.","content":"### Key Features of Dashboards\n\n1. **Panel Management**:\n   - Users can add, delete, and manage panels on a dashboard through both UI and Python API.  \n   - Various panel types include text panels, counters, line charts, pie charts, and bar charts—each customizable according to user needs.\n\n2. **Customization Options**:\n   - Dashboards can have multiple tabs to organize panels effectively. Users can create custom tabs and select from pre-built templates.  \n   - When adding panels, users specify metrics, panel types, titles, sizes, and plot parameters from a dropdown menu.\n\n3. **Incorporating Metrics**:\n   - Metrics must be defined when configuring panels. This can include direct metrics like `RowCount`, or more complex metrics requiring 

You: stop
